In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [5]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [6]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [7]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [8]:
payload = payload_json

In [9]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

The following field in agents.yaml appears more than once: hit_type_agent -> address -> alertedParty.inputRecordHist.inputRecords.INPUT_FIELD.ADDRESS1_COUNTRY
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE 

## transform standardized to cleansed

In [11]:
payload = pipeline.transform_standardized_to_cleansed(payload)

> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> 
ipdb> c
> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> c


ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [12]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [13]:
len(new_payloads)

2

In [14]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS']

0    {'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB']...
1    {'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB']...
Name: AP_TRIGGERS, dtype: object

In [15]:
out_payload[[i for i in out_payload.columns if "CARRIER" in i ]].values

array([['Doe'],
       ['Doe']], dtype=object)

In [16]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US|FL;US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [17]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,"[Joe BlackJ, Janusz]","[Joe BlackJ, Janusz]","[{'name': ['PARTY1_NAME_FULL', 'CONCAT_NAME'],..."
1,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."


In [18]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [10]:
with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [11]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [12]:
payload = payload_json

In [13]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

The following field in agents.yaml appears more than once: hit_type_agent -> address -> alertedParty.inputRecordHist.inputRecords.INPUT_FIELD.ADDRESS1_COUNTRY
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE 

## transform standardized to cleansed

In [14]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [15]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [16]:
len(new_payloads)

9

In [17]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['WL_MATCHED_TOKENS'], out_payload['TOKENS_RISK_CARRIER']

(0                           ["abouhamza", "abouhamza"]
 1    ["abdalrahmanmilad", "abdalrahmanmilad", "abda...
 2                           ["abdalhadi", "abdalhadi"]
 3                                      ["abdussomada"]
 4                                        ["abouhamza"]
 5                                    ["1177627035211"]
 6                                         ["ahmedoua"]
 7                                ["alfadhli", "udaia"]
 8                                      ["abdussomada"]
 Name: WL_MATCHED_TOKENS, dtype: object,
 0           abouhamza
 1    abdalrahmanmilad
 2           abdalhadi
 3         abdussomada
 4           abouhamza
 5       1177627035211
 6            ahmedoua
 7      alfadhli,udaia
 8         abdussomada
 Name: TOKENS_RISK_CARRIER, dtype: object)

In [18]:
out_payload[[i for i in out_payload.columns if "MATCH" in i ]].values

array([['["abouhamza", "abouhamza"]', 'N'],
       ['["abdalrahmanmilad", "abdalrahmanmilad", "abdalrahmanmilad"]',
        'N'],
       ['["abdalhadi", "abdalhadi"]', 'N'],
       ['["abdussomada"]', 'N'],
       ['["abouhamza"]', 'N'],
       ['["1177627035211"]', 'Y'],
       ['["ahmedoua"]', 'N'],
       ['["alfadhli", "udaia"]', 'N'],
       ['["abdussomada"]', 'N']], dtype=object)

In [19]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
94590752
0 match
[] ['1971', '01', '29', '01/29/1971']
['AHMEDOUA'] ['FR', 'FRANCE']
PAYLOAD
94590752
0 match
[] ['07/27/1986', '07', '27', '1986']
['AHMEDOUA'] ['LY', 'LIBYAN ARAB JAMAHIRIYA']
PAYLOAD
94590752
0 match
[] ['07', '1977', '06', '07/06/1977']
['AHMEDOUA'] []
PAYLOAD
94590752
0 match
[] ['1938', '08/17/1938', '08', '17']
['AHMEDOUA'] ['INDONESIA', 'ID']
PAYLOAD
94590752
0 match
[] ['15', '04', '04/15/1958', '1958']
['AHMEDOUA'] ['UNITED STATES', 'US']
PAYLOAD
94590752
0 match
[] []
['AHMEDOUA'] ['RUSSIAN FEDERATION', 'RU|PR. LENINA, D. 179']
PAYLOAD
94590752
0 match
[] ['01/01/1982', '01', '1982']
['AHMEDOUA'] ['MALI|MALI', 'ML|ML']
PAYLOAD
94590752
0 match
[] ['1981', '08/27/1981', '08', '27']
['AHMEDOUA'] []
PAYLOAD
94590752
0 match
[] ['1938', '08/17/1938', '08', '17']
['AHMEDOUA'] ['INDONESIA', 'ID']


In [20]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
1,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
2,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
3,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
4,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
5,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
6,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
7,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."
8,[],[],"[{'name': [], 'address': ['ADDRESS1_STATE', 'A..."


In [21]:
import pickle
with open("tests/shared/big_fat_payload_parsed.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [39]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [40]:
out_payload['TRIGGERED_BY']  = out_payload['TRIGGERED_BY'].apply(sorted)
reference_payload['TRIGGERED_BY']  = reference_payload['TRIGGERED_BY'].apply(sorted)


NameError: name 'reference_payload' is not defined

In [41]:
out_payload['AP_TRIGGERS']  = out_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})
reference_payload['AP_TRIGGERS']  = reference_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})


NameError: name 'reference_payload' is not defined

In [42]:
for cols in out_payload.columns:
    try:
        pd.testing.assert_series_equal(out_payload[cols], reference_payload[cols])
    except (AssertionError, TypeError):
        reference = remove_nulls_from_aggegated(flatten([i for i in reference_payload[cols].values]))
        output = remove_nulls_from_aggegated(flatten([i for i in out_payload[cols].values]))
        try:
            if isinstance(output, list):
                assert_list(output, reference)
            else:
                assert output == reference
        except AssertionError:
            print((cols, output, reference))
            pass

NameError: name 'reference_payload' is not defined

In [43]:
while True:
    print("remove", columns[-1])
    columns = columns[:-1]
    try:
        pd.testing.assert_frame_equal(out_payload[columns], reference_payload[columns])
        print('ok')
        break
    except AssertionError:
        break
        pass
    except TypeError:
        pass

NameError: name 'columns' is not defined

In [44]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
1,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
2,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
3,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
4,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
5,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
6,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
7,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
8,[],[],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."


In [45]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [46]:
payload = payload_json

In [47]:
payload = pipeline.transform_standardized_to_cleansed(payload)

> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> 
ipdb> c
> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> c
> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ip

In [48]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [49]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
1,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
2,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."


In [50]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
123
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US|FL;US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US|FL;US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [51]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [52]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [53]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [54]:
payload = payload_json

In [55]:
payload = pipeline.transform_standardized_to_cleansed(payload)

> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> 
ipdb> 
ipdb> c
> /app/etl_pipeline/custom/ms/watchlist_extractor/__init__.py(208)extract_country()
    206     def extract_country(self, match):
    207         import pdb; pdb.set_trace()
--> 208         try:
    209             address = match.get("entity", {}).get("addresses", {}).get("address")
    210         except AttributeError:

ipdb> c


In [56]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [57]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'address': ['CONCAT_ADDRESS', 'A..."


In [58]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
0 match
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [59]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
from etl_pipeline.config import load_agent_config

In [ ]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

In [ ]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

In [ ]:
import pickle

In [ ]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [ ]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [ ]:
from google.protobuf.any_pb2 import Any

In [ ]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [ ]:
i.feature_inputs[0].agent_feature_input

In [ ]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

In [ ]:
message

In [ ]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


In [ ]:
NameFeatureInput.ParseFromString()

In [ ]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

In [ ]:
type(i.feature_inputs[0].agent_feature_input.type_url)

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 